In [1]:
import pymysql
import pandas as pd

In [3]:
db = pymysql.connect("localhost", "root", "linlei", port=13306, charset='utf8')

In [4]:
cursor = [db.cursor()]

In [5]:
def sql(s):
    cursor[-1].execute(s)
    rst = cursor[-1].fetchall()
    
    rst = pd.DataFrame(rst, columns=[*zip(*cursor[-1].description)][0] if cursor[-1].description else [])
    return rst

In [6]:
def describe (table):
    rst = sql(f'describe {table};')
    rst.columns.name = table
    return rst

In [7]:
def select (items, table, *args, **kwargs):
    def deal (s):
        s = str(s).strip()
        return f'`{s}`' if s.isalnum() else s
    
    
    if type(items) is str:
        items = items.split(',')
    item = ','.join([deal(x) for x in items])
        
    if args:
        r = range(*args)
        start, stop = r.start, r.stop
        limit = f'limit {start}, {stop - start}'
    else: limit = ''
    
    s = f'select {item} from {deal(table)} {limit} {" ".join([f"{x} {kwargs[x]}" for x in kwargs if kwargs[x].strip()])}'
    
    rst = sql(s)
    rst.columns.name = table
    return rst

In [8]:
def count (table, *args, **kwargs):
    rst = select('count(*)', table, *args, **kwargs)
    return rst

In [33]:
c = cursor[0]

In [10]:
sql('use beihang')

""


In [16]:
sql('use academic_data;')

""


In [17]:
sql('show tables;')

,Tables_in_academic_data


In [15]:
sql('flush tables;')
count('paper')

paper,count(*)
0,31130


In [9]:
sql('flush tables;')
sql('show databases')

,Database
0,academic_data
1,beihang
2,information_schema
3,mysql
4,performance_schema
5,sys
6,test


In [25]:
sql('show databases;')

show databases;,Database
0,academic_data
1,information_schema
2,mysql
3,performance_schema
4,sys


In [18]:
db.close()